**Notebook objective**: 
1. Load graph object and NER data
2. Create graph object with and without node features

**Details**
- Graph object comes from ``functional_session_hit_directed_graph_er.gpickle'' file, i.e. the original graph file for the ER user journey.
- Two types of graph objects *with* features* are created: one with purely NER features and one with one-hot encoded nodes (to compensate for the fact that many nodes have zero feature vectors).

# Load packages

In [19]:
# Load packages
from src import functions as f
import pandas as pd
import networkx as nx

# Code

In [23]:
# Load data (original graph object and transition and adjacency matrices extracted from it)
G, A, T = f.get_er_graph("./data/inputs/functional_session_hit_directed_graph_er.gpickle")
ner_df = pd.read_csv("./data/inputs/ner_onehot.csv")

C:\Users\jbryb\virual_envs_python\pytorch_env\lib\site-packages\networkx\linalg\graphmatrix.py:187: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  return adjacency_matrix(G, nodelist, dtype, weight)
C:\Users\jbryb\OneDrive\Documents\Projects\github_gov_newtork_jrad\src\functions.py:45: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  T = nx.adjacency_matrix(G, weight="edgeWeight").todense()
C:\Users\jbryb\OneDrive\Documents\Projects\github_gov_newtork_jrad\src\functions.py:52: RuntimeWarning: invalid value encountered in divide
  T_probs = T_array / sum_of_rows[:, np.newaxis]  # increase dimension for broadcasting


Max row sum of probs: 1.0
Max row sum of probs: 1.0


C:\Users\jbryb\AppData\Local\Temp\ipykernel_21284\2313523018.py:3: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  ner_df = pd.read_csv("./data/inputs/ner_onehot.csv")


In [24]:
# clean NER data

# ner_df.head()
ner_df.drop(['Unnamed: 0', 'Unnamed: 1'], axis = 1, inplace = True) #.columns
ner_df["Unnamed: 2"] = ner_df["Unnamed: 2"].str.replace('https://www.gov.uk' , '')
ner_df.dropna(subset = ["Unnamed: 2"], inplace = True)
ner_df.rename(columns = {"Unnamed: 2": "page"}, inplace = True)
ner_df.set_index("page", inplace = True)
ner_df.drop('page_link \\ word index', inplace = True)
ner_df.head()

C:\Users\jbryb\AppData\Local\Temp\ipykernel_21284\468134769.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  ner_df["Unnamed: 2"] = ner_df["Unnamed: 2"].str.replace('https://www.gov.uk' , '')


,page_index_10 years,page_index_5 years,page_index_abroad,page_index_academics,page_index_academies,page_index_academy,page_index_academy trust,page_index_acas,page_index_accident,page_index_accommodation,...,page_index_world health organization,page_index_writing,page_index_year,page_index_years,page_index_young people,page_index_young person,page_index_youth,page_index_youth court,page_index_youth mobility scheme,page_index_zimbabwe
page,,,,,,,,,,,,,,,,,,,,,
/view-prove-immigration-status,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/prove-right-to-work,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/find-a-job,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/check-state-pension/sign-in/prove-identity,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
/jobseekers-allowance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:
print("G is directed:", G.is_directed())
print("Number of nodes of the graph:", G.number_of_nodes())

G is directed: True
Number of nodes of the graph: 10631


In [26]:
# G_from_T_features = G_from_T.copy()

In [27]:
G_from_T = get_graph_from_transition_matrix(T, list(G.nodes) )

In [28]:
print("G is directed:", G_from_T.is_directed())

G is directed: True


In [29]:
G_features, matrix_features, omitted_nodes, omitted_nodes_index = add_ner_features(G_from_T, ner_df)
G_features_wonehot, matrix_features_onehot = add_onehot_features(G_features, matrix_features, omitted_nodes_index)

In [48]:
print("Number of nodes with no NER features:", len(omitted_nodes), "i.e.", 
      np.round(100* len(omitted_nodes)/len(G_features.nodes() ),1), "%."
     )

Number of nodes with no NER features: 4651 i.e. 43.7 %.


In [49]:
G_features.nodes["/"]['node_feature'].shape

(1555,)

In [50]:
G_features_wonehot.nodes["/"]['node_feature'].shape

(6206,)

In [46]:
# Write or loaf objects as pickles 

# Write to pickle basiv graph matrices
f.dump_pickle_file(G, "Graph_er", dir_path="./data/graph_objects")
f.dump_pickle_file(G_from_T, "Graph_from_T", dir_path="./data/graph_objects")
f.dump_pickle_file(G_features, "Graph_er_weighted_wfeatures", dir_path="./data/graph_objects")
f.dump_pickle_file(G_features_wonehot, "Graph_er_weighted_wfeatures_onehot", dir_path="./data/graph_objects")
f.dump_pickle_file(A, "Adjacency_er", dir_path="./data/graph_objects")
f.dump_pickle_file(T, "Transition_er", dir_path="./data/graph_objects")

# Now load from pickles using a function in src folder
# G = f.load_pickle_file("Graph_er", dir_path="./data/graph_objects")
# G_from_T = f.load_pickle_file("Graph_from_T", dir_path="./data/graph_objects")
# A2 = f.load_pickle_file("Adjacency_er", dir_path="./data/graph_objects")
# T2 = f.load_pickle_file("Transition_er", dir_path="./data/graph_objects")

()